In [49]:
import networkx as nx
import pickle
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from itertools import combinations, groupby
import datetime

https://stackoverflow.com/questions/40088042/networkx-get-the-distance-between-nodes


In [50]:
entries = os.listdir('grafi/')

In [51]:
def load_graph(name):
    # directory = os.getcwd()
    # filename1 = directory + "/grafi/graph_V7500_E45000_weighted.pickle"

    # G = pickle.load(open(filename1, 'rb'))
    G = pickle.load(open("grafi/"+name, 'rb'))
    num_of_nodes = G.number_of_nodes()
    num_of_edges = G.number_of_edges()

    s = (num_of_nodes, num_of_edges)
    graph = np.zeros(s) # matrix of zeros

    print("num_of_nodes: ", num_of_nodes)
    print("len(graph): ", len(graph))

    for source, target in G.edges():
        graph[source][target] = G[source][target]['weight']

    return graph

# Edmonds-Karp algorithm

In [52]:
#Edmonds-Karp Algorithm
def edmonds_karp(C, s, t):
        n = len(C) # C is the capacity matrix
        F = [[0] * n for i in range(n)]
        path = bfs(C, F, s, t)
        print("BFS TROVATA")
      #  print path
        while path != None:
            flow = min(C[u][v] - F[u][v] for u,v in path)
            print("flow: ", flow)
            for u,v in path:
                F[u][v] += flow
                F[v][u] -= flow
            path = bfs(C, F, s, t)
        return sum(F[s][i] for i in range(n))

#find path by using BFS
def bfs(C, F, s, t):
        # print("cerco un nuovo path")

        queue = [s]
        paths = {s:[]}
        if s == t:
            return paths[s]
        while queue: 
            u = queue.pop(0)
            for v in range(len(C)):
                    if(C[u][v]-F[u][v]>0) and v not in paths:
                        paths[v] = paths[u]+[(u,v)]

                        last_key = list(paths)[-1]
                        #print(last_key)  # 👉️ country

                        # print(paths[last_key]) # 👉️ Belgium
                        # print("\n")
                        if v == t:
                            return paths[v]
                        queue.append(v)
        return None
    

# source = 0  # LA SORGENTE E' 1
# sink = num_of_nodes-1    # LA DESTINAZIONE E' 7
# max_flow_value = max_flow(graph, source, sink)
# print("Edmonds-Karp algorithm")
# print("max_flow_value is: ", max_flow_value)


# Dinitz Algorithm

In [53]:
#Dinic Algorithm

#build level graph by using BFS
def Bfs(C, F, s, t):  # C is the capacity matrix
        n = len(C)
        queue = []
        queue.append(s)
        global level
        level = n * [0]  # initialization
        level[s] = 1  
        while queue:
            k = queue.pop(0)
            for i in range(n):
                    if (F[k][i] < C[k][i]) and (level[i] == 0): # not visited
                            level[i] = level[k] + 1
                            queue.append(i)
        return level[t] > 0

#search augmenting path by using DFS
def Dfs(C, F, k, cp):
        tmp = cp
        if k == len(C)-1:
            return cp
        for i in range(len(C)):
            if (level[i] == level[k] + 1) and (F[k][i] < C[k][i]):
                f = Dfs(C,F,i,min(tmp,C[k][i] - F[k][i]))
                F[k][i] = F[k][i] + f
                F[i][k] = F[i][k] - f
                tmp = tmp - f
        return cp - tmp

#calculate max flow
#_ = float('inf')
def dinitz(C,s,t):
        n = len(C)
        F = [n*[0] for i in range(n)] # F is the flow matrix
        flow = 0
        while(Bfs(C,F,s,t)):
               flow = flow + Dfs(C,F,s,100000)
        return flow

# source = 0  # A
# sink = num_of_nodes-1    # F
# print("Dinic's Algorithm")
# max_flow_value = MaxFlow(graph, source, sink)
# print("max_flow_value is", max_flow_value)

# Ford-Fulkerson algorithm

In [54]:
#Ford-Fulkerson Algorithm

#find path by using BFS
def dfs(C, F, s, t):
        stack = [s]
        paths={s:[]}
        if s == t:
                return paths[s]
        while(stack):
                u = stack.pop()
                for v in range(len(C)):
                        if(C[u][v]-F[u][v]>0) and v not in paths:
                                paths[v] = paths[u]+[(u,v)]
                                if v == t:
                                        return paths[v]
                                stack.append(v)
        return None

def ford_fulkerson(C, s, t):
        n = len(C) # C is the capacity matrix
        F = [[0] * n for i in range(n)]
        path = dfs(C, F, s, t)
        while path != None:
            flow = min(C[u][v] - F[u][v] for u,v in path)
            for u,v in path:
                F[u][v] += flow
                F[v][u] -= flow
            path = dfs(C,F,s,t)
        return sum(F[s][i] for i in range(n))


# source = 0  # A
# sink = num_of_nodes-1    # F
# max_flow_value = max_flow(graph, source, sink)
# print("Ford-Fulkerson algorithm")
# print("max_flow_value is: ", max_flow_value)


# Capacity scaling algorithm

# Esecuzione degli algoritmi

In [55]:
def run_edmonds_karp(graph, source, sink):
    max_flow_value = edmonds_karp(graph, source, sink)
    print("Ford-Fulkerson algorithm")
    print("max_flow_value is: ", max_flow_value)

def run_dinic(graph, source, sink):
    max_flow_value = dinitz(graph, source, sink)
    print("Dinic's Algorithm")
    print("max_flow_value is", max_flow_value)

def run_ford_fulkerson(graph, source, sink):
    max_flow_value = ford_fulkerson(graph, source, sink)
    print("Ford-Fulkerson algorithm")
    print("max_flow_value is: ", max_flow_value)

In [56]:
# for i in entries:
print("Grafo corrente: ", 'graph_V5000_E40000_weighted.pickle')
graph = load_graph('graph_V5000_E40000_weighted.pickle')

num_of_nodes = len(graph)

source = num_of_nodes-2  # A
sink = num_of_nodes-1    # F

#ek
start = datetime.datetime.now()

run_edmonds_karp(graph, source, sink)

end = datetime.datetime.now()
delta = start - end
print("Tempo di esecuzione: ", delta.total_seconds())

#dinitz
start = datetime.datetime.now()

run_dinic(graph, source, sink)

end = datetime.datetime.now()
delta = start - end
print("Tempo di esecuzione: ", delta.total_seconds())

#ff
start = datetime.datetime.now()

run_ford_fulkerson(graph, source, sink)

end = datetime.datetime.now()
delta = start - end
print("Tempo di esecuzione: ", delta.total_seconds())

print("\n")

Grafo corrente:  graph_V5000_E40000_weighted.pickle
num_of_nodes:  5002
len(graph):  5002
BFS TROVATA
flow:  32.0
flow:  38.0
flow:  16.0
flow:  43.0
flow:  77.0
flow:  44.0
flow:  60.0
flow:  31.0
flow:  27.0
flow:  20.0
flow:  70.0
flow:  34.0
flow:  34.0
flow:  9.0
flow:  41.0
flow:  8.0
flow:  25.0
flow:  93.0
flow:  64.0
flow:  18.0
flow:  3.0
flow:  28.0
flow:  23.0
flow:  87.0
flow:  52.0
flow:  16.0
flow:  69.0
flow:  3.0
flow:  3.0
flow:  16.0
flow:  69.0
flow:  70.0
flow:  45.0
flow:  36.0
flow:  52.0
flow:  3.0
flow:  26.0
flow:  69.0
flow:  10.0
flow:  23.0
flow:  55.0
flow:  39.0
flow:  78.0
flow:  38.0
flow:  38.0
flow:  11.0
flow:  15.0
flow:  17.0
flow:  65.0
flow:  28.0
flow:  84.0
flow:  20.0
flow:  7.0
flow:  10.0
flow:  40.0
flow:  2.0
flow:  63.0
flow:  81.0
flow:  2.0
flow:  95.0
flow:  21.0
flow:  58.0
flow:  32.0
flow:  22.0
flow:  66.0
flow:  21.0
flow:  1.0
flow:  49.0
flow:  2.0
flow:  3.0
flow:  18.0
flow:  40.0
flow:  46.0
flow:  14.0
flow:  18.0
flow:  24.

In [ ]:
# for i in range(0, len(algorithms)):
#     print("Algoritmo: ", algorithms[i])
#     print("Tempo di esecuzione: ", times[i])
#     print("Flusso massimo: ", max_flow[i])
#     print("Taglio minimo: ", min_cut[i])
#     print("--------------------------------------------------")

In [ ]:
# >>> import datetime
# >>> a = datetime.datetime.now()
# >>> b = datetime.datetime.now()
# >>> delta = b - a
# >>> print delta
# 0:00:05.077263
# >>> int(delta.total_seconds() * 1000) # milliseconds
# 5077